In [1]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/lib/spark/"
os.environ["PYSPARK_PYTHON"]= "/home/cloudera/anaconda2/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]= "/home/cloudera/anaconda2/bin/python"
os.environ["SPARK_YARN_USER_ENV"]= "/home/cloudera/anaconda2/bin/python"

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.1-src.zip'))
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

from pyspark import SparkContext, SparkConf

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.0
      /_/

Using Python version 2.7.13 (default, Dec 20 2016 23:09:15)
SparkContext available as sc, HiveContext available as sqlContext.


##### The Kaggle Instatcart Market Basket analysis has data with more than 2 million rows in orders_product_prior and other datat set also has around a milloin rows that requires manipulation using partition and parallel processing.
##### For this reason In am using spark in jupyter and building my complete project in pycharm IDE. 
##### Created a virtual enviroment with spark libraries and anaconda python 2.7, initial settings are shown inthe first cell. 
##### Changed the 'dfs.blocksize' paramemter in the hdfs-default.xml to 64X1024X1024 bytes. 
##### This will increase parallisum in the hdfs processing for aggregating dataset in memory becomes faster.
##### for more information check this blog http://www.bigsynapse.com/spark-input-output.
##### Though I am using Pseudorandom distribution on cloudera quick start VM 5.8 it might not have any performance benifit as expected as machine is single but one cn try increasing the no of core and the executor memory.
##### default driver memory given is 5g in my VM , since in local mode executor is inside the driver so can't get memory more than that.##### underline properties to configure is spark  --num-executors <is one as installation is standalone> --executor-cores <depends on core of processor> --executor-memory <Memory available to the VM>

#### Order Schema 
##### order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_orde
##### read data with partition of three since data size of orders file is more than 110 mb we use 4 partition of size 3*32 mb
##### 32 mb is the local file system block size in whicj data is stored
##### 128 mb block size is the size of memory the name node referrences more 32 size blockes can be added to it.
##### It is for the namenode to decide in which 128 mb block refrence the 32 mb block can go

In [6]:
orders = sc.textFile("file:///home/cloudera/Desktop/kaggle/orders.csv", 4).filter(lambda x: 'order_id' not in x).map(lambda x : x.split(','))

In [90]:
orders.count()

3421083

#### Order_Product_Schema
##### order_id, Product_id, 
##### Here for we are taking the partition of 'order_products__prior' to be 18 as it is of more 550 mb 
##### similarly for 'order_products__train'  it is  default 0 partiton as the size is less than block size 32 mb

In [10]:
orders_prior = sc.textFile("file:///home/cloudera/Desktop/kaggle/order_products__prior.csv", 18).filter(lambda x: 'order_id' not in x).\
    map(lambda x : x.split(','))

orders_train = sc.textFile("file:///home/cloudera/Desktop/kaggle/order_products__train.csv").filter(lambda x: 'order_id' not in x).\
    map(lambda x : x.split(','))

In [21]:
orders_prior.take(1)

[[u'2', u'33120', u'1', u'1']]

##### Save all rdds created to hadoop file system in distributed fashion based on the no of the partions inthe rdd while loading 

In [23]:
orders.map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/orders")

In [24]:
orders_prior.map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/orders_prior")

In [25]:
orders_train.map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/orders_train")

In [28]:
## Define null prser to parse the blank strings 
def nullparse(x):
    if x =='':
        x=0
    else:
        float(x)
    return x

In [41]:
x= sc.textFile("/user/cloudera/kaggle/orders").map(lambda x: x.split(',')).map(lambda x : (int(x[0]),(x[1],x[2],x[3],x[4],x[5],str(nullparse(x[6])))))

In [38]:
y=sc.textFile("/user/cloudera/kaggle/orders_prior").map(lambda x: x.split(',')).map(lambda x : (int(x[0]), x[1]))

In [65]:
z= sc.textFile("/user/cloudera/kaggle/orders_train").map(lambda x: x.split(',')).map(lambda x : (int(x[0]), x[1]))

In [43]:
x.filter(lambda x : x[1][1]!="test").mapValues(lambda x : x[5]).reduceByKey(lambda x,y : x+y).\
    sortBy(lambda x : -x[1]).take(3)

##### Combine all products purchased for a perticular order id in one string sperated by space for order_product_prior data set.

In [45]:
y.groupByKey().mapValues(lambda x : ' '.join(x)).take(3)

[(3145734, u'45223 32740 46616 8230 6975 47668 3582'),
 (2621448, u'37718 38544 2748 47209 45504 28699 8006 41665 2078 35887'),
 (2097162, u'48988 37774 24852 19057')]

##### Save the intermediate result to the HDFS to reduce the time for further agregations

In [52]:
y.groupByKey().mapValues(lambda x : ' '.join(x)).map(lambda x : (str(x[0]),x[1])).map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/order_prior_inter")

In [53]:
order_prod_prio_inter = sc.textFile("/user/cloudera/kaggle/order_prior_inter").map(lambda x: x.split(','))

In [54]:
order_prod_prio_inter.take(2)

[[u'3145734', u'45223 32740 46616 8230 6975 47668 3582'],
 [u'2621448', u'37718 38544 2748 47209 45504 28699 8006 41665 2078 35887']]

In [57]:
order_prod_prio_inter.map(lambda x : (int(x[0]),x[1])).take(3)

[(3145734, u'45223 32740 46616 8230 6975 47668 3582'),
 (2621448, u'37718 38544 2748 47209 45504 28699 8006 41665 2078 35887'),
 (2097162, u'48988 37774 24852 19057')]

In [58]:
x.join(order_prod_prio_inter.map(lambda x : (int(x[0]),x[1]))).take(3)

[(2883584,
  ((u'78882', u'prior', u'1', u'3', u'20', '0'), u'43860 12249 34862')),
 (786434,
  ((u'29232', u'prior', u'16', u'0', u'12', '4.0'),
   u'29734 20847 4812 5491 31433 30192 45045 49628 38192 34134 26790 6891 7952 37658 28204 5194 22825 26346 32743 1158 18200 39977')),
 (1572868, ((u'14938', u'prior', u'7', u'0', u'14', '3.0'), u'21903'))]

In [61]:
x.join(order_prod_prio_inter.map(lambda x : (int(x[0]),x[1]))).mapValues(lambda x : x[0]+(x[1],)).map(lambda x : (str(x[0]), x[1])).\
map(lambda x : (x[0],) + x[1]).map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/order_product_prior_joined")

####order_product_prior_join Schema
##### order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order, all_purch_prod 


In [64]:
sc.textFile("/user/cloudera/kaggle/order_product_prior_joined").map(lambda x : x.split(',')).take(2)

[[u'2883584',
  u'78882',
  u'prior',
  u'1',
  u'3',
  u'20',
  u'0',
  u'43860 12249 34862'],
 [u'786434',
  u'29232',
  u'prior',
  u'16',
  u'0',
  u'12',
  u'4.0',
  u'29734 20847 4812 5491 31433 30192 45045 49628 38192 34134 26790 6891 7952 37658 28204 5194 22825 26346 32743 1158 18200 39977']]

In [75]:
z.groupByKey().mapValues(lambda x : ' '.join(x)).take(3)

[(1572864,
  u'19204 39698 48290 32000 40463 20682 13176 11526 33890 5025 14161 9387 13931 33198 43123 45351 5618'),
 (1, u'49302 11109 10246 49683 43633 13176 47209 22035'),
 (2359297, u'21781 4421 39009 36431 30233 5097 47209 28427 2513')]

##### Now lets create joined data set for order_product_train and the orders, this wont take much time as train data set is small and we can filter the orders data set for train only  

In [77]:
x.filter(lambda x : x[1][1]=='train').join(z.groupByKey().mapValues(lambda x : ' '.join(x))).mapValues(lambda x : x[0]+(x[1],)).map(lambda x : (str(x[0]), x[1])).\
map(lambda x : (x[0],) + x[1]).map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/order_product_train_joined")

#### order_product_train_join Schema
##### order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order, all_purch_prod

In [81]:
sc.textFile("/user/cloudera/kaggle/order_product_train_joined").take(3)

[u'1441800,23471,train,6,2,16,30.0,21713 24852 28204 3804 25544 8645 10916 29487 28634 17008',
 u'2621450,161225,train,25,4,13,30.0,46061',
 u'1703950,50594,train,4,4,12,30.0,19029 23594 8224 26882 18135 39922 38419 7042']

In [82]:
joined_train = sc.textFile("/user/cloudera/kaggle/order_product_train_joined")

In [83]:
joined_prior =  sc.textFile("/user/cloudera/kaggle/order_product_prior_joined")

In [89]:
joined_prior.map(lambda x : x.split(',')).map(lambda x : (int(x[0], int(x[1],int))))

0

In [93]:
subset_rdd_prior= sc.parallelize(joined_prior.take(10000))

In [106]:
subset_rdd_prior.take(2)

[u'2883584,78882,prior,1,3,20,0,43860 12249 34862',
 u'786434,29232,prior,16,0,12,4.0,29734 20847 4812 5491 31433 30192 45045 49628 38192 34134 26790 6891 7952 37658 28204 5194 22825 26346 32743 1158 18200 39977']

#### Combined Prior Order History Grouped by user
##### Now we have the combined data set based on the user from which we have to find insights 
##### Combined schema is a below grouped by user giving usera complete buying history 
##### user_id, all_order_id, order_numbers, all_order_dow, days_since_prior_orders, all_purch_prod_per_user

In [114]:
joined_prior.map(lambda x : x.split(',')).map(lambda x : (int(x[1]), (x[0], x[3],x[4],x[6],x[7] ))).\
    combineByKey((lambda x : ((x[0],), (x[1],),(x[2],),(x[3],), (x[4],))), 
                 (lambda x,y: (x[0] + (y[0],),x[1] + (y[1],),x[2] + (y[2],),x[3] + (y[3],),x[4] + (y[4],))),
                 (lambda x,y : (x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3],x[4]+y[4]))).take(1)

[(65538,
  ((u'2886687',
    u'2295459',
    u'2982586',
    u'3279300',
    u'1458602',
    u'2531850',
    u'1564378',
    u'1357315',
    u'848742',
    u'2174616',
    u'1726630',
    u'1847851',
    u'622275',
    u'2094779',
    u'2102018',
    u'1983262',
    u'2105407',
    u'1383433',
    u'1410274',
    u'499386',
    u'609586',
    u'1524281',
    u'740091',
    u'929490',
    u'941612',
    u'1063186',
    u'3158598',
    u'764140',
    u'1384101',
    u'476843',
    u'3115083',
    u'2509776',
    u'1878663',
    u'1153279',
    u'3041583',
    u'1888147',
    u'322077',
    u'2397932',
    u'2691303',
    u'3106399'),
   (u'15',
    u'28',
    u'1',
    u'24',
    u'27',
    u'30',
    u'35',
    u'8',
    u'36',
    u'13',
    u'31',
    u'5',
    u'32',
    u'34',
    u'21',
    u'37',
    u'18',
    u'16',
    u'40',
    u'17',
    u'26',
    u'9',
    u'7',
    u'38',
    u'23',
    u'14',
    u'33',
    u'29',
    u'6',
    u'22',
    u'11',
    u'4',
    u'25',
    

##### Taking the subsetted data for 1000 users for processing

In [116]:
sub_user= sc.parallelize(joined_prior.map(lambda x : x.split(',')).map(lambda x : (int(x[1]), (x[0], x[3],x[4],x[6],x[7] ))).\
    combineByKey((lambda x : ((x[0],), (x[1],),(x[2],),(x[3],), (x[4],))), 
                 (lambda x,y: (x[0] + (y[0],),x[1] + (y[1],),x[2] + (y[2],),x[3] + (y[3],),x[4] + (y[4],))),
                 (lambda x,y : (x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3],x[4]+y[4]))).take(1000))

In [117]:
sub_user.persist()  ### persist in memory for fast processing

ParallelCollectionRDD[349] at parallelize at PythonRDD.scala:423

In [157]:
def tuplesort(a,b,c,d,e):
    combined = list()
    x,y,z,s,t=(),(),(),(),()
    for i in range(len(b)):
        combined.append((a[i],int(b[i]),c[i],d[i],e[i]))    
    j= sorted(combined, key= lambda x : x[1])
    for i in range(len(combined)):
        x,y,z,s,t= x + (j[i][0],), y+(j[i][1],), z+ (j[i][2],), s + (j[i][3],), t +(j[i][4],)
    return [x,y,z,s,t]

##### order the data set based on order no.
##### user_id, all_order_id, order_numbers, all_order_dow, days_since_prior_orders, all_purch_prod_per_user
##### Data for one user in sorted order by order no 

In [167]:
sub_user.map(lambda x : (x[0], tuplesort(x[1][0], x[1][1], x[1][2],x[1][3],x[1][4])))\
    .map(lambda x : (x[0],' '.join(x[1][0]), ' '.join(x[1][2]),' '.join(x[1][3]),'|'.join(x[1][4]))).take(1)

[(65538,
  u'2982586 3106399 2397932 2509776 1847851 1384101 740091 1357315 1524281 322077 3115083 1888147 2174616 1063186 2886687 1383433 499386 2105407 2691303 1153279 2102018 476843 941612 3279300 1878663 609586 1458602 2295459 764140 2531850 1726630 622275 3158598 2094779 1564378 848742 1983262 929490 3041583 1410274',
  u'0 3 6 1 5 5 5 3 0 2 4 0 0 0 5 0 2 3 0 0 6 1 3 2 0 3 1 6 3 1 0 6 2 0 0 4 2 0 0 5',
  u'0 24.0 3.0 9.0 4.0 7.0 14.0 5.0 11.0 30.0 9.0 3.0 7.0 28.0 12.0 9.0 23.0 8.0 4.0 0.0 6.0 2.0 2.0 6.0 5.0 10.0 5.0 5.0 11.0 12.0 6.0 6.0 10.0 5.0 7.0 4.0 5.0 5.0 14.0 5.0',
  u'1511 18523 9934 47209 44473 4920 13740 40910 21290 21137 44275 49235 28204 4793 33497 10895 41950 25072 48230 28993 5194 38313 42383 15700 5077 39921 2450 35108 31263 43789 21903 41544 18610 49174 21461 30336 39928 37220 17979|1511 21137 5194 2770 44275 4920 13740 47209 6631 38313 25072 4793 28204 21290 895 38453 32293 33135 39928 24841 24561 17484|24964 22935 43076 19519 45535 38274|1511 21137 5194 4920 4

In [173]:
sub_user.unpersist()

ParallelCollectionRDD[349] at parallelize at PythonRDD.scala:423

In [168]:
inter_join=joined_prior.map(lambda x : x.split(',')).map(lambda x : (int(x[1]), (x[0], x[3],x[4],x[6],x[7] ))).\
    combineByKey((lambda x : ((x[0],), (x[1],),(x[2],),(x[3],), (x[4],))), 
                 (lambda x,y: (x[0] + (y[0],),x[1] + (y[1],),x[2] + (y[2],),x[3] + (y[3],),x[4] + (y[4],))),
                 (lambda x,y : (x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3],x[4]+y[4])))

##### Running a general recommendor engine based on colaborative filtering 

In [170]:
inter1_join=inter_join.map(lambda x : (x[0], tuplesort(x[1][0], x[1][1], x[1][2],x[1][3],x[1][4])))\
    .map(lambda x : (x[0],' '.join(x[1][0]), ' '.join(x[1][2]),' '.join(x[1][3]),'|'.join(x[1][4]))).\
    map(lambda x : (x[0], x[4]))

In [171]:
inter1_join.persist()

PythonRDD[366] at RDD at PythonRDD.scala:43

In [172]:
inter1_join.take(1)

[(65538,
  u'1511 18523 9934 47209 44473 4920 13740 40910 21290 21137 44275 49235 28204 4793 33497 10895 41950 25072 48230 28993 5194 38313 42383 15700 5077 39921 2450 35108 31263 43789 21903 41544 18610 49174 21461 30336 39928 37220 17979|1511 21137 5194 2770 44275 4920 13740 47209 6631 38313 25072 4793 28204 21290 895 38453 32293 33135 39928 24841 24561 17484|24964 22935 43076 19519 45535 38274|1511 21137 5194 4920 44275 33497 9934 38313 28204 25072 4793 44473 895 40910 2452 39921 22170 20327 18610 7806 2081 4659 47766 17484 39781|49235 10749 44359 39275 21137 26209 28842 33746 3376|1511 21137 5194 49235 44275 44473 6631 2452 38313 40910 895 39921 22170 20327 10132 24841 39928 9076 21543 12204 3481 47766 17484 36772 18610 1896|2770 47209 34023 28849 3376 1999 5818 19678 31343 16570 39928 28204 12254 38274|2770 1511 21137 5194 44275 47209 44473 40910 28204 2452 38313 895 25072 22170 39921 33135 3481 18610 18523 35108 36036 16093 48720 21019 1896 26344|34551 3481 35108 24841 1896 26344

In [176]:
prod_chain = "1511 18523 9934 47209 44473 4920 13740 40910 21290 21137 44275 49235 28204 4793 33497 10895 41950 25072 48230 28993 5194 38313 42383 15700 5077 39921 2450 35108 31263 43789 21903 41544 18610 49174 21461 30336 39928 37220 17979|1511 21137 5194 2770 44275 4920 13740 47209 6631 38313 25072 4793 28204 21290 895 38453 32293 33135 39928 24841 24561 17484|24964 22935 43076 19519 45535 38274|1511 21137 5194 4920 44275 33497 9934 38313 28204 25072 4793 44473 895 40910 2452 39921 22170 20327 18610 7806 2081 4659 47766 17484 39781|49235 10749 44359 39275 21137 26209 28842 33746 3376|1511 21137 5194 49235 44275 44473 6631 2452 38313 40910 895 39921 22170 20327 10132 24841 39928 9076 21543 12204 3481 47766 17484 36772 18610 1896|2770 47209 34023 28849 3376 1999 5818 19678 31343 16570 39928 28204 12254 38274|2770 1511 21137 5194 44275 47209 44473 40910 28204 2452 38313 895 25072 22170 39921 33135 3481 18610 18523 35108 36036 16093 48720 21019 1896 26344|34551 3481 35108 24841 1896 26344 38641 21461 35045 49174 9808 1700 12980 6873 39430 14267|4920 4793 21137 44275 38313 1896 38641 26344|34448 22888 10761 9595 48230 32293 22935 45535 4920 44359 10132 1511 21137 22170|13740|37029 48230 2450 49235 5194|9598 4138 10761 42383 13740 24561 22935 4920 44359 10132 1511 44275 2452 895 28204 3481 1896 26344|9934 49235 39928 1511 28204|28985 32175 12916 45066 26790 12144 13740 4793 42265 39928 1511 44275 18610 35108|26790 13740 38453 4920 44359 47766 42265 1511 44275 44473 2452 1896 26344|4920 49235 1511|40604 34126 22474 4138 33198 44375 41220 27104 24964 31343 1511 25072 18610 35108 21461 49174|37119 22888 44375 10895 48230 28993 2450 32293 1511 22170 39921 18610 35108 12980|44912|41220 27104 24964 1511 21461 49174|22474 15143 44375 10761 13740 49235 25072 18610|18200 38383 39561 12144 37220 13740 44359 47766 17484 42265 1511 2770 5194 25072 22170 24841|45066 24852 2770 20995 31343 1999 18479 38562|1511 47766 13176 49235 2770 44275 28204 19048 581 34448 3599 26790 16145 8670 47626"

In [198]:
def listcounter(list):
    from collections import Counter
    counts = Counter(list)
    return counts.items()

##### count frequency of each product purchased by the user to impement recommendations using collaborative filter techniques. 

In [238]:
user_product_matrix = inter1_join.mapValues(lambda x : x.split("|")).mapValues(lambda x : ' '.join(x)).mapValues(lambda x : x.split(' ')).\
    map(lambda x: (x[0],listcounter(x[1]))).map(lambda x : [(x[0],)+ i for i in x[1]]).flatMap(lambda x : x).\
    map(lambda x : (str(x[0]),str(x[1]),str(x[2])) ).map(lambda x : ' '.join(x)).map(lambda x : x.split(' ')).\
    map(lambda x : [int(y) for y in x])

In [239]:
user_product_matrix.persist()

PythonRDD[548] at RDD at PythonRDD.scala:43

In [240]:
user_product_matrix.take(10)

[[65538, 44359, 7],
 [65538, 9076, 1],
 [65538, 25072, 10],
 [65538, 47209, 6],
 [65538, 31343, 3],
 [65538, 21903, 1],
 [65538, 48416, 1],
 [65538, 34126, 1],
 [65538, 22935, 4],
 [65538, 22170, 7]]

In [224]:
inter1_join.unpersist()

PythonRDD[366] at RDD at PythonRDD.scala:43

In [229]:
from pyspark.mllib.recommendation import Rating, ALS

In [241]:
als_matrix =user_product_matrix.map(lambda x : Rating(x[0], x[1], x[2]))

##### Compute user rating rdd for training data set to find the model performance  

In [258]:
model = ALS.trainImplicit(als_matrix , 10, 10, 0.01)

In [260]:
model.recommendProducts(1, 15)

[Rating(user=1, product=13176, rating=0.6833920201082382),
 Rating(user=1, product=12341, rating=0.2606711224255599),
 Rating(user=1, product=6184, rating=0.25304948680000483),
 Rating(user=1, product=43352, rating=0.23523993177616165),
 Rating(user=1, product=21903, rating=0.21333270362711204),
 Rating(user=1, product=21137, rating=0.1810016485532501),
 Rating(user=1, product=196, rating=0.16125479473361332),
 Rating(user=1, product=42265, rating=0.1545010261115591),
 Rating(user=1, product=35561, rating=0.1408376275833716),
 Rating(user=1, product=39275, rating=0.13990913621452966),
 Rating(user=1, product=49235, rating=0.1329687481167136),
 Rating(user=1, product=21288, rating=0.1319131345745536),
 Rating(user=1, product=8048, rating=0.12885602575585925),
 Rating(user=1, product=30827, rating=0.12618212607377818),
 Rating(user=1, product=16797, rating=0.12424837651376963)]

In [280]:
model.save(sc, "/user/cloudera/kaggle/model")

In [281]:
user_product_matrix.unpersist()

PythonRDD[548] at RDD at PythonRDD.scala:43

##### Compute user rating rdd for training data set to find the model performance  

In [249]:
train_data = joined_train.map(lambda x : x.split(',')).map(lambda x : (int(x[1]), (x[0], x[3],x[4],x[6],x[7] ))).\
    combineByKey((lambda x : ((x[0],), (x[1],),(x[2],),(x[3],), (x[4],))), 
                 (lambda x,y: (x[0] + (y[0],),x[1] + (y[1],),x[2] + (y[2],),x[3] + (y[3],),x[4] + (y[4],))),
                 (lambda x,y : (x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3],x[4]+y[4]))).map(lambda x : (x[0], tuplesort(x[1][0], x[1][1], x[1][2],x[1][3],x[1][4])))\
    .map(lambda x : (x[0],' '.join(x[1][0]), ' '.join(x[1][2]),' '.join(x[1][3]),'|'.join(x[1][4]))).map(lambda x : (x[0], x[4])).mapValues(lambda x : x.split("|")).mapValues(lambda x : ' '.join(x)).mapValues(lambda x : x.split(' ')).\
    map(lambda x: (x[0],listcounter(x[1]))).map(lambda x : [(x[0],)+ i for i in x[1]]).flatMap(lambda x : x).\
    map(lambda x : (str(x[0]),str(x[1]),str(x[2])) ).map(lambda x : ' '.join(x)).map(lambda x : x.split(' ')).\
    map(lambda x : [int(y) for y in x]).map(lambda x : [x[0],x[1]])

In [252]:
train_data.persist()

PythonRDD[825] at RDD at PythonRDD.scala:43

In [261]:
predictions = model.predictAll(train_data).map(lambda r: ((r[0], r[1]), r[2]))

In [263]:
predictions.take(50)

[((156013, 17601), 0.005849102692537208),
 ((156013, 24852), 0.7391857780478983),
 ((156013, 2966), 0.03360833882420569),
 ((103301, 37257), 0.005271245290309876),
 ((103301, 33731), 0.0875574633099883),
 ((48510, 517), 0.013002532566667564),
 ((48510, 46941), 0.07975798909229428),
 ((48510, 32004), 0.00490046676878498),
 ((48510, 13802), 0.05643948368854896),
 ((165737, 4357), 0.013270055249441626),
 ((165737, 13819), 0.024175879277026587),
 ((165737, 16019), 0.00469022200586823),
 ((165737, 8277), 0.20567527370262928),
 ((165737, 46997), 0.001177208391205721),
 ((165737, 22974), 0.002409096715833825),
 ((165737, 27509), 0.016921683066339444),
 ((165737, 1406), 0.004884045454793455),
 ((193259, 38159), 0.19364532383908023),
 ((193259, 28204), 0.2198832427118078),
 ((193259, 42736), 0.27837735499853056),
 ((193259, 40174), 0.08257852843897551),
 ((193259, 38777), 0.19355528914651984),
 ((193259, 22025), 0.08768430701952536),
 ((193259, 32255), 0.004062573058343427),
 ((193259, 25072), 

##### Seems like prediction ratings does not perform well on the training set
##### FInding the average rating to define the threshhold

In [272]:
predictions.map(lambda x : x[1]).reduce(lambda x,y : x+y)/predictions.map(lambda x : x[1]).count()

0.10464606522477252

##### since average rating for the training set 0.11 so product having rating greater than the average rating should be considered as the possible product

##### recommending product for the test set 

In [286]:
test_users= x.filter(lambda  x : x[1][1]=='test').map(lambda x : x[1][0]).distinct()

In [282]:
train_data.unpersist()

PythonRDD[825] at RDD at PythonRDD.scala:43

In [288]:
test_users.persist()

PythonRDD[1146] at RDD at PythonRDD.scala:43

In [291]:
userlist= test_users.map(lambda x : int(x))

[35540, 73469, 98848, 35544, 146054, 73461, 53001, 161057, 145208, 11542]

In [292]:
quit()  # closing spark shell